<h1>The Battle of Neighborhood | Part 2</h1>

<h3>Import Libraries</h3>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import requests # library to handle requests
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
kuala_lumpur_df = pd.read_csv('Kuala_Lumpur.csv')
kuala_lumpur_df.head(10)

,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148499,101.696728
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610
5,Bandar Tun Razak,3.082760,101.722810
6,Bangsar,3.129200,101.678440
7,Bangsar Park,3.129200,101.678440
8,Bangsar South,3.111020,101.662830
9,Batu 11 Cheras,3.061870,101.746750


<h3>Create a Map of Kuala Lumpur with neighborhood superimposed on top</h3>

In [3]:
# Get the coordinates of 'Kuala Lumpur'
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Kuala Lumpur, Malaysia are {a} and {b}'.format(a= latitude, b= longitude))

The geographical coordinates of Kuala Lumpur, Malaysia are 3.1516964 and 101.6942371


In [4]:
# Create Map of Kuala Lumpur, Malaysia using latitude and longitude values
map_KualaLumpur = folium.Map(location= [latitude, longitude], zoom_start=11)

# Add markers to map
for lat, lng, neighborhood in zip(kuala_lumpur_df['Latitude'], kuala_lumpur_df['Longitude'], kuala_lumpur_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker([lat, lng],
                        radius= 5,
                        popup= label,
                        color= 'blue',
                        fill= True,
                        fill_color= '#3186cc',
                        fill_opacity= 0.7).add_to(map_KualaLumpur)
    
map_KualaLumpur

In [5]:
# Save the map as HTML file
map_KualaLumpur.save('Kuala_Lumpur_Neighborhood.html')

<h3>Use the Foresquare API to explore the neighborhoods</h3>

In [6]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'L3XLTZ2HARAD1BNWETR0JZYF3C3K0HYGEV4I3MV1A30EOSEF' # your Foursquare ID
CLIENT_SECRET = 'QDHJGUKOJQ02VRG3HNQWS3PE3NJFFOIIPAEWTN4XEEGPIAED' # your Foursquare Secret
VERSION = '20180605'

print('Your credentails:\n')
print('CLIENT_ID: ' + CLIENT_ID)
print('\nCLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:

CLIENT_ID: L3XLTZ2HARAD1BNWETR0JZYF3C3K0HYGEV4I3MV1A30EOSEF

CLIENT_SECRET: QDHJGUKOJQ02VRG3HNQWS3PE3NJFFOIIPAEWTN4XEEGPIAED


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [7]:
radius = 2000
LIMIT = 100

venues = []

for lat, lng, neighborhood in zip(kuala_lumpur_df['Latitude'], kuala_lumpur_df['Longitude'], kuala_lumpur_df['Neighborhood']):
    
    # Create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
    
    # Make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    # Return only relevant information for each nearby venue
    for venue in results:
        venues.append((neighborhood, lat, lng, 
                       venue['venue']['name'], 
                       venue['venue']['location']['lat'], 
                       venue['venue']['location']['lng'],
                       venue['venue']['categories'][0]['name']))

In [8]:
# Convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# Define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(10)

(7038, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant
2,Alam Damai,3.05769,101.74388,日日生饮食中心,3.057629,101.749410,Restaurant
3,Alam Damai,3.05769,101.74388,Restaurant Sweetland 甜园美食中心,3.057495,101.747725,Chinese Restaurant
4,Alam Damai,3.05769,101.74388,Leng Leng 靓靓茶餐室,3.054049,101.749902,Food Court
5,Alam Damai,3.05769,101.74388,Food Truck Lake Valley,3.046220,101.750686,Food Truck
6,Alam Damai,3.05769,101.74388,Aj Maju,3.046632,101.739467,Indian Restaurant
7,Alam Damai,3.05769,101.74388,Ivy Sekinchan Seafood Noodle House 适耕莊特制魚丸海鲜面,3.065749,101.748718,Noodle House
8,Alam Damai,3.05769,101.74388,SUSHI MENTAI,3.057821,101.748145,Japanese Restaurant
9,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House


**Let's check how many venues were returned for each neighorhood**

In [9]:
venues_df.groupby(['Neighborhood']).count().head(10)

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,94,94,94,94,94,94
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,97,97,97,97,97,97
Bandar Tasik Selatan,94,94,94,94,94,94
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


**Let's find out how many unique categories can be created from all the returned venues**

In [10]:
print('There are {} unique categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 287 unique categories.


In [11]:
# Print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Supplement Shop', 'Chinese Restaurant', 'Restaurant',
       'Food Court', 'Food Truck', 'Indian Restaurant', 'Noodle House',
       'Japanese Restaurant', 'Bubble Tea Shop', 'Asian Restaurant',
       'Coffee Shop', 'Convenience Store', 'Pet Store', 'Farmers Market',
       'Middle Eastern Restaurant', 'Mamak Restaurant', 'Breakfast Spot',
       'Dim Sum Restaurant', 'Park', 'Other Great Outdoors',
       'Malay Restaurant', 'Café', 'Gym / Fitness Center',
       'Padangnese Restaurant', 'Fast Food Restaurant', 'Burger Joint',
       'Winery', 'Badminton Court', 'Gas Station', 'Burrito Place',
       'Halal Restaurant', 'Cantonese Restaurant', 'Steakhouse',
       'College Bookstore', 'Bistro', 'Thai Restaurant',
       'Seafood Restaurant', 'Flea Market', 'Salon / Barbershop',
       'Hostel', 'Dessert Shop', 'Exhibit', 'South Indian Restaurant',
       'Kebab Restaurant', 'Chettinad Restaurant', 'Bed & Breakfast',
       'Gym', 'Speakeasy', 'Museum', 'Monument / Landmark'],

In [12]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

<h3>Analyze Each  Neighborhood</h3>

In [13]:
# One hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood']

# Move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(7038, 288)


,Neighborhoods,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Betting Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Business Service,Butcher,Cafeteria,Café,Cantonese Restaurant,Caribbean Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Padangnese Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Print Shop,Pub,Racetrack,Ramen Restaurant,Rehab Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tree,Udon Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Veter

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [14]:
kl_grouped = kl_onehot.groupby(['Neighborhoods']).mean().reset_index()

print(kl_grouped.shape)
kl_grouped.head(20)

(71, 288)


,Neighborhoods,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Betting Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Business Service,Butcher,Cafeteria,Café,Cantonese Restaurant,Caribbean Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Padangnese Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Print Shop,Pub,Racetrack,Ramen Restaurant,Rehab Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tree,Udon Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Veter

In [15]:
len(kl_grouped[kl_grouped['Shopping Mall'] > 0])

27

**Create a new DataFrame for Shopping Mall data only**

In [16]:
kl_mall = kl_grouped[['Neighborhoods', 'Shopping Mall']]

print(kl_mall.shape)
kl_mall.head(10)

(71, 2)


,Neighborhoods,Shopping Mall
0,Alam Damai,0.00
1,"Ampang, Kuala Lumpur",0.01
2,Bandar Menjalara,0.00
3,Bandar Sri Permaisuri,0.00
4,Bandar Tasik Selatan,0.00
5,Bandar Tun Razak,0.00
6,Bangsar,0.02
7,Bangsar Park,0.02
8,Bangsar South,0.02
9,Batu 11 Cheras,0.00


<h3>Cluster  Neighborhoods</h3>
<p>Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 cluster.</p>

In [17]:
# Set number of cluster
kcluster = 3

kl_clustering = kl_mall.drop(labels=['Neighborhoods'], axis=1)

# Run k-means clustering
kmeans = KMeans(n_clusters= kcluster, random_state=0).fit(kl_clustering)

# Check Cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 0, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 0])

In [18]:
# Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# Add Clustering labels column
kl_merged['Cluster Labels'] = kmeans.labels_

# Rename 'Neighborhoods' column as 'Neighborhood'
kl_merged.rename(columns= {'Neighborhoods':'Neighborhood'}, inplace= True)
kl_merged.head(10)

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.00,1
1,"Ampang, Kuala Lumpur",0.01,0
2,Bandar Menjalara,0.00,1
3,Bandar Sri Permaisuri,0.00,1
4,Bandar Tasik Selatan,0.00,1
5,Bandar Tun Razak,0.00,1
6,Bangsar,0.02,2
7,Bangsar Park,0.02,2
8,Bangsar South,0.02,2
9,Batu 11 Cheras,0.00,1


In [19]:
# Merge kuala_lumpur_df with kl_merge to add latitude and longitude for each neighborhood
kl_merged = kl_merged.join(kuala_lumpur_df.set_index('Neighborhood'), on= 'Neighborhood')

print(kl_merged.shape)
kl_merged.head(10)       # check the last two columns!

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,1,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.01,0,3.148499,101.696728
2,Bandar Menjalara,0.00,1,3.190350,101.625450
3,Bandar Sri Permaisuri,0.00,1,3.103910,101.712260
4,Bandar Tasik Selatan,0.00,1,3.072750,101.714610
5,Bandar Tun Razak,0.00,1,3.082760,101.722810
6,Bangsar,0.02,2,3.129200,101.678440
7,Bangsar Park,0.02,2,3.129200,101.678440
8,Bangsar South,0.02,2,3.111020,101.662830
9,Batu 11 Cheras,0.00,1,3.061870,101.746750


In [20]:
# Sort the results by cluster labels
kl_merged.sort_values(by=['Cluster Labels'], inplace= True)

print(kl_merged.shape)
kl_merged.head(10)

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
70,Wangsa Maju,0.01,0,3.20387,101.73715
30,Kampung Datuk Keramat,0.01,0,3.16640,101.73046
40,Mont Kiara,0.01,0,3.16529,101.65242
45,Segambut,0.01,0,3.18650,101.66795
47,"Sentul, Kuala Lumpur",0.01,0,3.17508,101.69305
20,Damansara Heights,0.01,0,3.14797,101.66795
19,Chow Kit,0.01,0,3.16378,101.69814
50,Shamelin,0.01,0,3.12457,101.73597
56,Taman Desa,0.01,0,3.10297,101.68471
51,Sri Hartamas,0.01,0,3.16220,101.65036


**Finally, let's visualize the resulting clusters**

In [21]:
# Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Set color scheme for the clusters
x  = np.arange(kcluster)
ys = [i+x+(i*x)**2 for i in range(kcluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lng, neighborhood, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(neighborhood) + ' : Cluster ' + str(cluster), parse_html= True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup= label,
                        color= rainbow[cluster-1],
                        fill= True,
                        fill_color= rainbow[cluster-1],
                        fill_opacity= 0.7).add_to(map_clusters)
    
map_clusters    

In [22]:
# Save the map as HTML file
map_clusters.save('KualaLumpur_map_clusters.html')

<h2>Examine Cluster</h2>
<h4>Cluster 0</h4>

In [24]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
70,Wangsa Maju,0.01,0,3.203870,101.737150
30,Kampung Datuk Keramat,0.01,0,3.166400,101.730460
40,Mont Kiara,0.01,0,3.165290,101.652420
45,Segambut,0.01,0,3.186500,101.667950
47,"Sentul, Kuala Lumpur",0.01,0,3.175080,101.693050
20,Damansara Heights,0.01,0,3.147970,101.667950
19,Chow Kit,0.01,0,3.163780,101.698140
50,Shamelin,0.01,0,3.124570,101.735970
56,Taman Desa,0.01,0,3.102970,101.684710
51,Sri Hartamas,0.01,0,3.162200,101.650360


<h4>Cluster 1</h4>

In [25]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
58,Taman Ibukota,0.0,1,3.212310,101.715250
38,Medan Tuanku,0.0,1,3.159260,101.698340
39,Miharja,0.0,1,3.147890,101.694050
68,Taman Wahyu,0.0,1,3.222400,101.671730
41,Pantai Dalam,0.0,1,3.094760,101.667470
43,Putrajaya,0.0,1,3.125860,101.718624
44,Salak South,0.0,1,3.081540,101.696890
65,Taman Taynton View,0.0,1,3.087070,101.736810
46,Semarak,0.0,1,3.179943,101.721449
64,Taman Sri Sinar,0.0,1,3.190070,101.652930


<h4>Cluster 2</h4>

In [26]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
36,Lembah Pantai,0.02,2,3.121189,101.663889
67,Taman U-Thant,0.03,2,3.157700,101.724520
66,Taman Tun Dr Ismail,0.03,2,3.152830,101.622710
31,Kampung Kasipillay,0.02,2,3.177760,101.682400
29,"Kampung Baru, Kuala Lumpur",0.02,2,3.165460,101.710280
6,Bangsar,0.02,2,3.129200,101.678440
28,KL Eco City,0.02,2,3.117130,101.673840
8,Bangsar South,0.02,2,3.111020,101.662830
57,Taman Duta,0.02,2,3.155620,101.671840
11,Brickfields,0.03,2,3.129160,101.684060


#### Observations:
Most of the shopping malls are concentrated in the central area of Kuala Lumpur city, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.